<a href="https://colab.research.google.com/github/Reemaalt/Detection-of-Hallucination-in-Arabic/blob/main/RougeL_Labeling_of_Answers_XOR_ISRI-clean.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##instal

In [9]:
!pip install rouge-score

#Test RougeL

In [ ]:
from huggingface_hub import login
login()

In [11]:
!pip install nltk

#Use isri stemmer and jais tokenizer

In [12]:
from transformers import AutoTokenizer
from rouge_score import rouge_scorer
from nltk.stem.isri import ISRIStemmer  # Import ISRI stemmer

# Load tokenizer
model_name = "core42/jais-13b"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Initialize ISRI Stemmer
stemmer = ISRIStemmer()

# Define a custom tokenizer class that applies both tokenization and stemming
class CustomTokenizer:
    def __init__(self, tokenizer, stemmer):
        self.tokenizer = tokenizer
        self.stemmer = stemmer

    def tokenize(self, text):
        words = text.split()  # Split into words first
        stemmed_words = [self.stemmer.stem(word) for word in words]  # Apply stemming
        print("Stemmed Words:", stemmed_words)  # Debugging line to check the stemmed words
        return self.tokenizer.tokenize(" ".join(stemmed_words))  # Tokenize the stemmed text

# Create an instance of our custom tokenizer
custom_tokenizer = CustomTokenizer(tokenizer, stemmer)

# Create the ROUGE scorer with our custom tokenizer
scorer = rouge_scorer.RougeScorer(['rougeL'], tokenizer=custom_tokenizer)

# Example strings
pred_str = 'السلام عليكم كيف حالك'
label_str = 'السلام عليكم صديقي كيف حالك'

# Compute ROUGE scores
scores = scorer.score(label_str, pred_str)

# Print the stemmed words to check if stemming occurs
for key, value in scores.items():
    print(f'{key}: {value}')


tokenizer_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/4.85M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Stemmed Words: ['سلم', 'علي', 'صدق', 'كيف', 'حلك']
Stemmed Words: ['سلم', 'علي', 'كيف', 'حلك']
rougeL: Score(precision=1.0, recall=0.8333333333333334, fmeasure=0.9090909090909091)


# combine partial results

In [5]:
import json

# Load the original JSON file (list of question dictionaries)
input_path = "/content/new_Labeled_SE_ALLaM_ArabicaQA.json"
output_path = "/content/questions_with_human_label_arabica.json"

# Load the data
with open(input_path, "r", encoding="utf-8") as file:
    data = json.load(file)

# Filter questions that contain the "human_question_label" field
filtered_data = [item for item in data if "human_question_label" in item]

# Save to a new JSON file
with open(output_path, "w", encoding="utf-8") as f:
    json.dump(filtered_data, f, ensure_ascii=False, indent=4)

print(f"Saved {len(filtered_data)} questions with 'human_question_label' to '{output_path}'")


Saved 100 questions with 'human_question_label' to '/content/questions_with_human_label_arabica.json'


#Use RougeL on data

In [13]:
import json




# Load data from a JSON file
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Save updated data to a new JSON file
def save_labels(output_data, output_file_path):
    with open(output_file_path, 'w', encoding='utf-8') as file:
        json.dump(output_data, file, ensure_ascii=False, indent=4)

# Process hallucination detection using ROUGE-L
def check_hallucination(data, rouge_threshold=0.3, min_non_hallucinated=6):
    for item in data:
        original_answer = item.get("original_answer", "").strip()
        non_hallucinated_count = 0

        updated_answers = []
        for cluster in item.get("answers", []):
            updated_cluster = []

            for answer in cluster:
                answer_text = answer.get("text", "").strip()

                # Compute ROUGE-L score
                scores = scorer.score(original_answer, answer_text)
                rouge_l_f1 = round(scores["rougeL"].fmeasure, 4)

                # Assign hallucination label
                label = "Non-Hallucinated" if rouge_l_f1 >= rouge_threshold else "Hallucinated"
                if label == "Non-Hallucinated":
                    non_hallucinated_count += 1

                # Wrap answer into a dictionary with ROUGE info
                answer_dict = {
                    "text": answer_text,
                    "RougeL_score": rouge_l_f1,
                    "RougeL_label": label
                }

                updated_cluster.append(answer_dict)

            updated_answers.append(updated_cluster)

        item["answers"] = updated_answers
        item["computed_question_label"] = (
            "Non-Hallucinated" if non_hallucinated_count >= min_non_hallucinated else "Hallucinated"
        )

    return data

# Main function
def main(input_file_path, output_file_path):
    data = load_data(input_file_path)
    updated_data = check_hallucination(data)
    save_labels(updated_data, output_file_path)
    print(f"Labeled data has been saved to {output_file_path}")

# Paths to your files
input_file_path = "/content/questions_with_human_label_arabica.json"
output_file_path = "humanArabica_rougel_isri.json"

# Run the labeling
main(input_file_path, output_file_path)


Stemmed Words: ['وسط']
Stemmed Words: ['لعب', 'دميترو', 'زوزول', 'في', 'ركز', 'وسط']
Stemmed Words: ['وسط']
Stemmed Words: ['لعب', 'دميترو', 'زوزول', 'في', 'ركز', 'وسط']
Stemmed Words: ['وسط']
Stemmed Words: ['لعب', 'دميترو', 'زوزول', 'في', 'ركز', 'خط', 'وسط', 'في', 'كرة', 'قدم']
Stemmed Words: ['وسط']
Stemmed Words: ['لعب', 'دميترو', 'زوزول', 'في', 'ركز', 'وسط', 'ايم']
Stemmed Words: ['وسط']
Stemmed Words: ['لعب', 'دميترو', 'زوزول', 'كجناح', 'يسر', 'في', 'كرة', 'قدم']
Stemmed Words: ['وسط']
Stemmed Words: ['لعب', 'دميترو', 'زوزول', 'كجناح', 'يسر', 'في', 'كرة', 'قدم']
Stemmed Words: ['وسط']
Stemmed Words: ['ركز', 'لعب', 'دميترو', 'زوزول', 'في', 'كرة', 'قدم', 'هو', 'في', 'ركز', 'هجم']
Stemmed Words: ['وسط']
Stemmed Words: ['لعب', 'دميترو', 'زوزول', 'في', 'ركز', 'هجم']
Stemmed Words: ['وسط']
Stemmed Words: ['لعب', 'دميترو', 'زوزول', 'كمهاجم', 'في', 'كرة', 'قدم']
Stemmed Words: ['وسط']
Stemmed Words: ['ركز', 'لعب', 'دميترو', 'زوزول', 'في', 'كرة', 'قدم', 'هو', 'هجم']
Stemmed Words: ['لعب',

In [7]:
import json

# Load your JSON file
with open("/content/finalTransWithSE_QwenXOR_rougel_isri.json", "r", encoding="utf-8") as f:
    data = json.load(f)

# Initialize counters
hallucinated_count = 0
non_hallucinated_count = 0

# Iterate through each question item in the list
for item in data:
    label = item.get("computed_question_label", "")
    if label == "Hallucinated":
        hallucinated_count += 1
    elif label == "Non-Hallucinated":
        non_hallucinated_count += 1

# Print results
print(f"Number of hallucinated questions: {hallucinated_count}")
print(f"Number of non-hallucinated questions: {non_hallucinated_count}")


Number of hallucinated questions: 11678
Number of non-hallucinated questions: 916


#evaluate rouge-l labeling against human labeling for a sample

In [18]:
import json

# Load data from JSON file
def load_data(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        return json.load(file)

# Compare computed vs human labels and break down accuracy by label type
def compare_labels(data):
    correct_non_hallucinated = 0
    correct_hallucinated = 0
    incorrect_non_hallucinated = 0
    incorrect_hallucinated = 0

    for item in data:
        human_label = item.get("human_question_label")
        computed_label = item.get("computed_question_label")

        if human_label == "Non-Hallucinated" and computed_label == "Non-Hallucinated":
            correct_non_hallucinated += 1
        elif human_label == "Hallucinated" and computed_label == "Hallucinated":
            correct_hallucinated += 1
        elif human_label == "Non-Hallucinated" and computed_label == "Hallucinated":
            incorrect_non_hallucinated += 1
        elif human_label == "Hallucinated" and computed_label == "Non-Hallucinated":
            incorrect_hallucinated += 1

    total = correct_non_hallucinated + correct_hallucinated + incorrect_non_hallucinated + incorrect_hallucinated
    accuracy = (correct_non_hallucinated + correct_hallucinated) / total if total > 0 else 0

    return {
        "correct_non_hallucinated": correct_non_hallucinated,
        "correct_hallucinated": correct_hallucinated,
        "incorrect_non_hallucinated": incorrect_non_hallucinated,
        "incorrect_hallucinated": incorrect_hallucinated,
        "accuracy": accuracy
    }

# File path to your labeled data
input_file_path = "/content/humanArabica_rougel_isri.json"

# Load data
data = load_data(input_file_path)

# Compare labels
results = compare_labels(data)

# Print detailed results
print("=== Label Comparison Report ===")
print(f" Correct Non-Hallucinated: {results['correct_non_hallucinated']}")
print(f" Correct Hallucinated: {results['correct_hallucinated']}")
print(f" Incorrect Non-Hallucinated: {results['incorrect_non_hallucinated']}")
print(f" Incorrect Hallucinated: {results['incorrect_hallucinated']}")
print(f"\n Overall Accuracy: {results['accuracy'] * 100:.2f}%")


=== Label Comparison Report ===
 Correct Non-Hallucinated: 8
 Correct Hallucinated: 53
 Incorrect Non-Hallucinated: 30
 Incorrect Hallucinated: 9

 Overall Accuracy: 61.00%


In [14]:
import json
import sys
from sklearn.metrics import accuracy_score, recall_score

def load_data(file_path):
    """Load JSON data from the provided file path."""
    try:
        with open(file_path, 'r', encoding='utf-8') as file:
            return json.load(file)
    except FileNotFoundError:
        print(f"Error: File not found at '{file_path}'")
        sys.exit(1)
    except json.JSONDecodeError:
        print("Error: Failed to decode JSON.")
        sys.exit(1)

def extract_labels(data):
    """Extract human and computed labels from the data."""
    human_labels = []
    computed_labels = []

    for item in data:
        human_label = item.get("human_question_label")
        computed_label = item.get("computed_question_label")

        # Only add if both labels are present
        if human_label is not None and computed_label is not None:
            human_labels.append(human_label)
            computed_labels.append(computed_label)

    return human_labels, computed_labels

def main():
    input_file_path = "/content/humanArabica_rougel_isri.json"
    data = load_data(input_file_path)
    y_true, y_pred = extract_labels(data)

    # Convert to binary format: Hallucinated = 1, Non-Hallucinated = 0
    label_to_int = lambda x: 1 if x == "Hallucinated" else 0
    y_true_binary = list(map(label_to_int, y_true))
    y_pred_binary = list(map(label_to_int, y_pred))

    # Compute metrics
    accuracy = accuracy_score(y_true_binary, y_pred_binary)
    recall = recall_score(y_true_binary, y_pred_binary)

    # Report
    print("=== Scikit-learn Evaluation Report ===")
    print(f" Accuracy: {accuracy * 100:.2f}%")
    print(f" Recall (Hallucinated class): {recall * 100:.2f}%")

if __name__ == "__main__":
    main()


=== Scikit-learn Evaluation Report ===
 Accuracy: 61.00%
 Recall (Hallucinated class): 85.48%
